In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8MB 217kB/s 
     |████████████████████████████████| 9.1MB 40.3MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 460kB 45.4MB/s 
     |████████████████████████████████| 1.1MB 24.0MB/s 
     |████████████████████████████████| 122kB 55.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=9447581048c5e4336fbf2b8e42f5a66d09fc991f41616946e274a79a967c9c21
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.0.0
    Uninstalling smart-open-5.0.0:
      Successfully uninstalled smart-open-5.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-

In [2]:
import torch, torchtext
from torch import nn, optim, functional as F
import pandas as pd, csv
import pdb
import random
import numpy as np

In [29]:
!unzip tracks.zip

Archive:  tracks.zip
  inflating: tracks.csv              


In [4]:
!wget -O text.csv https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0

--2021-05-13 02:44:54--  https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iulhdbo1yc8farq/Emotion_final.csv [following]
--2021-05-13 02:44:54--  https://www.dropbox.com/s/raw/iulhdbo1yc8farq/Emotion_final.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccfe36f373ed193a41947007440.dl.dropboxusercontent.com/cd/0/inline/BOb76fK9vTWskyMeI9XjMIxtwNe7pl6QjIL362AJGE3bioGbXd0Os1JqMdy44N7CaVrmeuhbahAKfROdjm3wSOWvmDMYII2U8OVYMbM9wm_y_0jrhgVYOWYHBz11ss2PaPCF-aHbDztVwLXhoPo0htM_/file# [following]
--2021-05-13 02:44:55--  https://uccfe36f373ed193a41947007440.dl.dropboxusercontent.com/cd/0/inline/BOb76fK9vTWskyMeI9XjMIxtwNe7pl6QjIL362AJGE3bioGbXd0Os1JqMdy44N7CaVrmeu

In [5]:
text = pd.read_csv('/content/text.csv')

In [6]:
sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

In [81]:
def remove_punctuation(text): #function for removing punctuation
    import string 
    translator = str.maketrans('', '', string.punctuation) #replace the punctuations with no space
    return text.translate(translator)

In [82]:
class Sentences(torch.utils.data.Dataset):
    def __init__(self, fn):
        lengths = []
        convert = { u: n for n, u in enumerate(fn['Emotion'].unique()) }
        fn['Emotion'] = fn['Emotion'].apply(lambda u: convert[u])               # 12 unique words should be assigned integers starting from 0
        tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')# tokenizer using spaCy
        for i in range(len(fn['Text'])):
          lengths.append(len(tokenizer(fn['Text'].iat[i].strip())))                   # store the number of tokens in each sentence to beused in get item
        string = ' '.join([fn['Text'].iat[i].strip() 
                           for i in range(len(fn['Text']))])                  # combine everything into one single string
        toks = tokenizer(string)                                                # tokenize the single string

        self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
        self.sentiment = fn['Emotion'].values
        self.text = fn['Text'].values
        self.length = lengths
        self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])

    def __len__(self):
        return len(self.length)

    def __getitem__(self, i):
        sum = 0
        for x in range(i):
          sum += self.length[x]
        return (self.sentiment[i], self.toks[sum: sum + self.length[i]])          # return the sentiment and related tokns for a specific tweet
    
    def input_to_tensor(self, string):
        string = remove_punctuation(string)
        in_toks = string.split(" ")
        return torch.LongTensor([self.vocab[i] for i in in_toks])

In [83]:
ds_full = Sentences(text)
n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

1lines [00:00, 23.58lines/s]


In [9]:
class SentenceModel(nn.Module):                                                 # takes in a sentence, and outputs predicted sentiment
      def __init__(self, vocab_size, embedding_dim, lstm_dim, 
                   n_cats, n_layers = 2, drop_prob = 0.5):
        super().__init__()                                                      #constructor for parent class
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)          #use word embeddings 
        self.lstm = torch.nn.LSTM(embedding_dim, lstm_dim, n_layers,
                                  dropout=drop_prob, batch_first=True)          #LSTM layer
        self.linear = nn.Linear(lstm_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.linear.weight.data)
        
      def forward(self, text):
        emb = self.embedding(text)
        lstm_out, _ = self.lstm(emb)
        out = self.linear(lstm_out)
        return torch.mean(out, dim=1)                                           # certain dimensions required so take mean to reduce them down

In [10]:
#First locate tracks.zip and paste the file path in the unzip below

# **DATASET CODE HERE**

upload -> sentence_model_state_dict (the saved fully trained model) 
---


In [84]:
model = SentenceModel(len(ds_full.vocab)+2, 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

In [85]:
# Model class must be defined somewhere
model.load_state_dict(torch.load("/content/sentence_model_state_dict.pth"))
model.eval()

SentenceModel(
  (embedding): Embedding(19910, 64)
  (lstm): LSTM(64, 64, num_layers=2, batch_first=True, dropout=0.5)
  (linear): Linear(in_features=64, out_features=6, bias=True)
)

In [86]:
ldr = torch.utils.data.DataLoader(ds_test) 

In [87]:
def get_weights(tensor=None, model=None): #tensor should have shape ([x, y, z, ...]) (1 dim), NOT ([[x, y, z, ...]]), the function itself unsqueezes the input tensor for you
  tensor = tensor.unsqueeze(0)
  device = torch.device('cuda:0')
  model.to(device)
  m = nn.ReLU()
  s = nn.Softmax(dim=1)
  with torch.no_grad():
    update_tensor = model(tensor.to(device))
    print(update_tensor)
    relud_logged = torch.log(m(update_tensor))
    print(relud_logged)
    print("Tensor converted to text: " + ' '.join([ds_full.vocab.itos[x] for x in tensor.squeeze()]))
    return s(relud_logged).squeeze()
  
  # this function calculates the appropriate probability distribution for every track based on input "tensor" and our model

In [88]:
tracks = pd.read_csv('/content/tracks.csv')
tracks.drop(inplace=True, columns=['duration_ms','key', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'time_signature', 'loudness'])
tracks.sort_values(by=['popularity'], ascending=False)

,id,name,popularity,explicit,artists,id_artists,release_date,danceability,energy,valence
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100,1,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.6960,0.464
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,1,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.4360,0.132
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98,0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.6950,0.472
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.8260,0.644
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97,0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.5240,0.553
...,...,...,...,...,...,...,...,...,...,...
23410,0sniGbmm1sjg51BxO6AHD4,O Pardesi Musafir,0,0,"['Suraiya', 'Lata Mangeshkar']","['71lk7UDkZK1mabUE6LCASW', '61JrslREXq98hurYL2...",1949-01-01,0.682,0.3050,0.910
23409,0sk8KA96i9MIU8St2nmbIx,Mere Dil Mein Aao Aur Bas Jao,0,0,['Suraiya'],['71lk7UDkZK1mabUE6LCASW'],1949-01-01,0.527,0.2030,0.688
23408,0sdkzsc3WtZIrSi1nNyl7v,"Diversions, Op. 21: Variation 9, Toccata I",0,0,"['Benjamin Britten', 'Leon Fleisher', 'Seiji O...","['7MJ1pB5d6Vjmzep2zQlorn', '6ncNdxBc8zVWMOF7nJ...",1949,0.441,0.2420,0.528
23407,0s6h3lMdJsWkpUAW6xC7m3,John Henry,0,0,['Lead Belly'],['3Ovf2lytXSXWFM2cwsJACC'],1949,0.593,0.7480,0.732


sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

In [89]:
# list_scores = []
# in = ('sadness', 'anger', 'love', 'surprise', 'fear', 'happy')
# for **track** in list of tracks over 65 popularity:
#    score = POPULARITY_WEIGHT * (track.popularity / 100) + in.sadness * (1 - track.valence) + in.anger * track.energy + in.love * (mean(track.energy + track.dancability)) + in.surprise * (1 - track.energy) + in.fear * valence + in.happy * (mean(track.danceability + track.energy))
#    list_scores.append(score)

# np.random.choice(tracks, list_scores)

In [90]:
new_df = tracks[tracks.popularity >=65]

In [91]:
def normalize(p): # makes sure the distribution probability list adds to 1
    if sum(p) != 1.0:
        p = np.asarray(p)*(1.0/sum(p))
    return p

In [92]:
#user input is x -> we have to convert user input string into tensor "x"
#so input would be "input = get_weights(x, model)"

POPULARITY_WEIGHT = 1.5 # can be changed 
NUM_TRACK_OPTIONS = 25 # can be changed

input_string = "i'm having a great day today!"
id_tracks = new_df["id"].values.tolist()
scores_id = []
seq = ds_full.input_to_tensor(input_string)
input = get_weights(seq, model)
for track in new_df.itertuples(index=False):
    score = POPULARITY_WEIGHT*(track.popularity/100) + input[0].item()*(1-track.valence) + input[1].item()*track.energy + input[2].item()*((track.energy + track.danceability)/2) + input[3].item()*(1-track.energy) + input[4].item()*track.valence + input[5].item()*((track.danceability + track.energy)/2)
    scores_id.append([score, track.id])

distribution = []
top_track_ids = []
s = sorted(scores_id, reverse=True)
s = s[0:NUM_TRACK_OPTIONS]

for score, id in s:
  distribution.append(score)
  top_track_ids.append(id)
print("\n")
id = np.random.choice(top_track_ids, p=normalize(distribution)) # picks a random track across a distribution generated by the scores
print("Distribution: {0}".format(input))
print("TRACK LINK: https://open.spotify.com/track/" + id)
actual_track = new_df[new_df['id']==id]
print(actual_track)

tensor([[ 1.0117,  2.0134, -2.9310,  1.4684, -0.3427, -1.9823]],
       device='cuda:0')
tensor([[0.0116, 0.6998,   -inf, 0.3841,   -inf,   -inf]], device='cuda:0')
Tensor converted to text: <unk> having a great day today


Distribution: tensor([0.2251, 0.4481, 0.0000, 0.3268, 0.0000, 0.0000], device='cuda:0')
TRACK LINK: https://open.spotify.com/track/65OVbaJR5O1RmwOQx0875b
                           id  ... valence
93823  65OVbaJR5O1RmwOQx0875b  ...     0.1

[1 rows x 10 columns]
